In [9]:
# -*- coding: utf-8 -*-
import os
import shutil
import datetime
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

import tensorflow as tf
from tensorflow.keras.models import load_model
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from sklearn.metrics import confusion_matrix, classification_report

# =========================
# 設定（必要に応じて変更）
# =========================
TEST_DIR   = r"C:\kanno\vscode\RNN-for-Human-Activity-Recognition-using-2D-Pose-Input-master\RNN-for-Human-Activity-Recognition-using-2D-Pose-Input-master\data\test_cnn"    # 例: data/test/<class_name>/*.png
MODEL_PATH = r"C:\kanno\vscode\RNN-for-Human-Activity-Recognition-using-2D-Pose-Input-master\RNN-for-Human-Activity-Recognition-using-2D-Pose-Input-master\scripts\models_cnn_traj\traj_cnn_20260108-131214.h5"  # 学習済みモデルのパス
IMG_SIZE   = (300, 300)
BATCH_SIZE = 32

# 誤分類画像をコピーで保存するかどうか
COPY_MISCLASSIFIED = True

# 出力ルート（タイムスタンプでサブフォルダ作成）
OUT_ROOT = "output"
now = datetime.datetime.now().strftime("%Y%m%d_%H%M%S")
EVAL_DIR = os.path.join(OUT_ROOT, f"eval_{now}")
CM_DIR   = os.path.join(EVAL_DIR, "confusion_matrix")
REP_DIR  = os.path.join(EVAL_DIR, "report")
MIS_DIR  = os.path.join(EVAL_DIR, "misclassified")
PRED_DIR = os.path.join(EVAL_DIR, "pred_csv")

os.makedirs(CM_DIR,  exist_ok=True)
os.makedirs(REP_DIR, exist_ok=True)
os.makedirs(MIS_DIR, exist_ok=True)
os.makedirs(PRED_DIR, exist_ok=True)

# =========================
# モデル読み込み
# =========================
print("[INFO] loading model:", MODEL_PATH)
model = load_model(MODEL_PATH)

# =========================
# テストデータジェネレータ
# =========================
datagen = ImageDataGenerator(rescale=1./255)
test_gen = datagen.flow_from_directory(
    TEST_DIR,
    target_size=IMG_SIZE,
    batch_size=BATCH_SIZE,
    class_mode='categorical',   # 多クラスにも対応
    shuffle=False               # ファイル名順を保つ（予測と紐付けしやすい）
)

class_indices = test_gen.class_indices             # {'classA':0, 'classB':1, ...}
class_labels  = [c for c,_ in sorted(class_indices.items(), key=lambda x: x[1])]
print("[INFO] classes:", class_labels)

# =========================
# 予測
# =========================
y_true = test_gen.classes                           # 0..C-1
filenames = test_gen.filenames                      # TEST_DIR からの相対パス
print("[INFO] #test images:", len(filenames))

y_prob = model.predict(test_gen, verbose=1)         # (N, C)
y_pred = np.argmax(y_prob, axis=1)                  # 予測ラベル index

# =========================
# 予測一覧CSV（ファイルごと）
# =========================
cols = ["file", "true_idx", "true_label", "pred_idx", "pred_label"]
for i, c in enumerate(class_labels):
    cols.append(f"p_{c}")

rows = []
for i, rel in enumerate(filenames):
    row = [rel, int(y_true[i]), class_labels[y_true[i]], int(y_pred[i]), class_labels[y_pred[i]]]
    row += [float(y_prob[i, j]) for j in range(len(class_labels))]
    rows.append(row)

df_pred = pd.DataFrame(rows, columns=cols)
pred_csv_path = os.path.join(PRED_DIR, f"predictions_{now}.csv")
df_pred.to_csv(pred_csv_path, index=False, encoding="utf-8-sig")
print(f"✅ 予測CSV: {pred_csv_path}")

# =========================
# 分類レポート
# =========================
report_txt = classification_report(y_true, y_pred, target_names=class_labels, digits=4)
rep_path = os.path.join(REP_DIR, f"classification_report_{now}.txt")
with open(rep_path, "w", encoding="utf-8") as f:
    f.write(report_txt + "\n")
print(f"✅ 分類レポート: {rep_path}")

# =========================
# 混同行列（生/正規化）PNG
# =========================
cm = confusion_matrix(y_true, y_pred, labels=list(range(len(class_labels))))
cm_png = os.path.join(CM_DIR, f"cm_raw_{now}.png")
cmn_png = os.path.join(CM_DIR, f"cm_normalized_{now}.png")

plt.figure(figsize=(6, 5))
sns.heatmap(cm, annot=True, fmt='d', cmap='Blues',
            xticklabels=class_labels, yticklabels=class_labels)
plt.xlabel("Predicted"); plt.ylabel("True"); plt.title("Confusion Matrix (Raw)")
plt.tight_layout(); plt.savefig(cm_png); plt.close()
print(f"✅ 混同行列(生)保存: {cm_png}")

cm_norm = cm.astype(np.float32) / np.clip(cm.sum(axis=1, keepdims=True), 1, None)
plt.figure(figsize=(6, 5))
sns.heatmap(cm_norm, annot=True, fmt='.3f', cmap='Blues',
            xticklabels=class_labels, yticklabels=class_labels, vmin=0, vmax=1)
plt.xlabel("Predicted"); plt.ylabel("True"); plt.title("Confusion Matrix (Row-Normalized)")
plt.tight_layout(); plt.savefig(cmn_png); plt.close()
print(f"✅ 混同行列(正規化)保存: {cmn_png}")

# =========================
# 誤分類のリスト/コピー
# =========================
mis_rows = df_pred[df_pred["true_idx"] != df_pred["pred_idx"]].copy()
mis_txt  = os.path.join(MIS_DIR, f"misclassified_list_{now}.txt")
mis_csv  = os.path.join(MIS_DIR, f"misclassified_list_{now}.csv")

with open(mis_txt, "w", encoding="utf-8") as f:
    for _, r in mis_rows.iterrows():
        f.write(f"True: {r['true_label']}, Predict: {r['pred_label']}, file: {r['file']}\n")
mis_rows.to_csv(mis_csv, index=False, encoding="utf-8-sig")
print(f"✅ 誤分類リスト: {mis_txt}")
print(f"✅ 誤分類CSV  : {mis_csv}")

if COPY_MISCLASSIFIED and len(mis_rows) > 0:
    # 予測クラス別にサブフォルダを切ってコピー
    for _, r in mis_rows.iterrows():
        src = os.path.join(TEST_DIR, r["file"])
        sub = os.path.join(MIS_DIR, f"pred_{r['pred_label']}_true_{r['true_label']}")
        os.makedirs(sub, exist_ok=True)
        dst = os.path.join(sub, os.path.basename(r["file"]))
        # ファイル名衝突回避
        base, ext = os.path.splitext(dst)
        k = 1
        while os.path.exists(dst):
            dst = f"{base}_{k}{ext}"
            k += 1
        try:
            shutil.copy2(src, dst)
        except Exception as e:
            print(f"[WARN] copy failed for {src}: {e}")
    print(f"✅ 誤分類画像コピー保存先: {MIS_DIR}")

print("\n[Done] Evaluation outputs saved under:", EVAL_DIR)


[INFO] loading model: C:\kanno\vscode\RNN-for-Human-Activity-Recognition-using-2D-Pose-Input-master\RNN-for-Human-Activity-Recognition-using-2D-Pose-Input-master\scripts\models_cnn_traj\traj_cnn_20260108-131214.h5
Found 80 images belonging to 2 classes.


c:\kanno\envs\MLya\Lib\site-packages\keras\src\trainers\data_adapters\py_dataset_adapter.py:121: UserWarning: Your `PyDataset` class should call `super().__init__(**kwargs)` in its constructor. `**kwargs` can include `workers`, `use_multiprocessing`, `max_queue_size`. Do not pass these arguments to `fit()`, as they will be ignored.
  self._warn_if_super_not_called()


[INFO] classes: ['ivdd', 'normal']
[INFO] #test images: 80
3/3 ━━━━━━━━━━━━━━━━━━━━ 1s 150ms/step
✅ 予測CSV: output\eval_20260108_140520\pred_csv\predictions_20260108_140520.csv
✅ 分類レポート: output\eval_20260108_140520\report\classification_report_20260108_140520.txt
✅ 混同行列(生)保存: output\eval_20260108_140520\confusion_matrix\cm_raw_20260108_140520.png
✅ 混同行列(正規化)保存: output\eval_20260108_140520\confusion_matrix\cm_normalized_20260108_140520.png
✅ 誤分類リスト: output\eval_20260108_140520\misclassified\misclassified_list_20260108_140520.txt
✅ 誤分類CSV  : output\eval_20260108_140520\misclassified\misclassified_list_20260108_140520.csv
[WARN] copy failed for C:\kanno\vscode\RNN-for-Human-Activity-Recognition-using-2D-Pose-Input-master\RNN-for-Human-Activity-Recognition-using-2D-Pose-Input-master\data\test_cnn\ivdd\two_case10DLC_resnet152_sotuken1Dec17shuffle1_150000_proc_leftpaw_0-60_norm.png: [WinError 3] 指定されたパスが見つかりません。
[WARN] copy failed for C:\kanno\vscode\RNN-for-Human-Activity-Recognition-using-2